1. Data Exploration and Preprocessing

In [13]:
#Load the Dataset
import pandas as pd
df = pd.read_csv('/content/adult_with_headers.csv')
df.info()
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


Handle Missing Values:

For numerical features, use imputation (mean/median) or removal based on the percentage of missing values.

In [2]:
#Check for missing values and data types.
df.isnull().sum()
df.dtypes


,0
age,int64
workclass,object
fnlwgt,int64
education,object
education_num,int64
marital_status,object
occupation,object
relationship,object
race,object
sex,object


In [3]:
df['age'].fillna(df['age'].median(), inplace=True)
df.dropna(subset=['education'], inplace=True)


Apply Scaling Techniques:

Standard Scaling: Standardize numerical features to have a mean of 0 and a standard deviation of 1.
python
Copy code


In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df['age_scaled'] = scaler.fit_transform(df[['age']])


In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['age_minmax'] = scaler.fit_transform(df[['age']])


2. Encoding Techniques

In [6]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income', 'age_scaled', 'age_minmax'],
      dtype='object')

In [7]:
df = pd.get_dummies(df, columns=['workclass', 'marital_status'], drop_first=True)


In [8]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['education_encoded'] = encoder.fit_transform(df['education'])


3. Feature Engineering

In [9]:
df['high_edu_hours'] = df['education'].apply(lambda x: 1 if x in ['Bachelors', 'Masters', 'Doctorate'] else 0) * df['hours_per_week']


In [10]:
import numpy as np
df['log_capital_gain'] = np.log1p(df['capital_gain'])


4. Feature Selection

In [11]:
from sklearn.ensemble import IsolationForest
iso_forest = IsolationForest(contamination=0.05)
outliers = iso_forest.fit_predict(df[['age', 'hours_per_week', 'capital_gain']])
df['outliers'] = outliers
df = df[df['outliers'] == 1]


PPS (Predictive Power Score):

In [12]:
!pip install ppscore
import ppscore as pps
pps_matrix = pps.matrix(df)
print(pps_matrix)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklea

             x                        y   ppscore                case  \
0          age                      age  1.000000      predict_itself   
1          age                   fnlwgt  0.000000          regression   
2          age                education  0.044648      classification   
3          age            education_num  0.000000          regression   
4          age               occupation  0.026606      classification   
...        ...                      ...       ...                 ...   
1084  outliers  marital_status_ Widowed  0.000000          regression   
1085  outliers        education_encoded  0.000000          regression   
1086  outliers           high_edu_hours  0.000000  target_is_constant   
1087  outliers         log_capital_gain  0.000000          regression   
1088  outliers                 outliers  1.000000      predict_itself   

      is_valid_score               metric  baseline_score   model_score  \
0               True                 None       